In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [4]:
# load the data set ans show the first five transaction
# Read CSV
url="https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv"
df= pd.read_csv(url , sep=',')
df.head(5)

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [5]:
items = set()
for col in df:
    if df[col].unique != " " :
        items.update(df[col].unique())
print(items)

{nan, 'Bagel', 'Diaper', 'Eggs', 'Pencil', 'Wine', 'Milk', 'Meat', 'Bread', 'Cheese'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [6]:
#create an itemset based on the products
itemset = set(items)

# encoding the feature
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]



{nan: 0,
 'Bagel': 0,
 'Milk': 0,
 'Diaper': 1,
 'Eggs': 1,
 'Pencil': 1,
 'Wine': 1,
 'Bread': 1,
 'Meat': 1,
 'Cheese': 1}

In [7]:
  # create new dataframe from the encoded features
ohe_df = pd.DataFrame(encoded_vals)
  # show the new dataframe
ohe_df.head(5)

,NaN,Bagel,Milk,Diaper,Eggs,Pencil,Wine,Bread,Meat,Cheese
0,0,0,0,1,1,1,1,1,1,1
1,0,0,1,1,0,1,1,1,1,1
2,1,0,1,0,1,0,1,0,1,1
3,1,0,1,0,1,0,1,0,1,1
4,1,0,0,0,0,1,1,0,1,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [8]:
ohe_df.drop(ohe_df.columns[[0]], axis=1, inplace=True)
ohe_df.head(5)

,Bagel,Milk,Diaper,Eggs,Pencil,Wine,Bread,Meat,Cheese
0,0,0,1,1,1,1,1,1,1
1,0,1,1,0,1,1,1,1,1
2,0,1,0,1,0,1,0,1,1
3,0,1,0,1,0,1,0,1,1
4,0,0,0,0,1,1,0,1,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [9]:
from mlxtend.frequent_patterns import apriori, association_rules
freq_items = apriori(ohe_df,use_colnames=True, min_support=0.2)
freq_items.head(7)

,support,itemsets
0,0.425397,(Bagel)
1,0.501587,(Milk)
2,0.406349,(Diaper)
3,0.438095,(Eggs)
4,0.361905,(Pencil)
5,0.438095,(Wine)
6,0.504762,(Bread)


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [10]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
2,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
3,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
4,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
